# Data Classes

First we need to import the `dataclass`-decorator

In [111]:
from dataclasses import dataclass

from datetime import datetime


Let's create a simple data class


In [112]:
@dataclass
class PyCon:
    location: str
    date: datetime
    year: int = 2018  # Default year

It's that simple, let's try it out

If we create an instance without specifying default a TypeError will be raised

In [113]:
try:
    PyCon()
except TypeError as e:
    print(e)

__init__() missing 2 required positional arguments: 'location' and 'date'


Location and date missing, the year is given the default value. We can see that the properties are translated to kwargs in the `__init__()`-function

Ok so let's try to properly instantiate it

In [114]:
# For you who are obesrvant, fromisoformat() was added to datetime in Python 3.7

pycon_se_18 = PyCon("Stockholm", datetime.fromisoformat("2018-12-12"))
pycon_se_18

PyCon(location='Stockholm', date=datetime.datetime(2018, 12, 12, 0, 0), year=2018)

That worked, but could be nicer.

Let's improve our class some
- Set the year to the current year instead of hardcoding
- Pass the date as a _iso-string_ an convert it automatically

In [124]:
from dataclasses import field
from typing import Union

@dataclass
class PyCon:
    location: str
    # Can be passed as a str or datetime
    date: Union[datetime, str]
    
    # Add a factory for this year
    year: int = field(
        default_factory=lambda: datetime.now().year
    )
    
    # post init runs after __init__ finishes, here we can add more nicities
    def __post_init__(self):
        
        if isinstance(self.date, str):
            # Will raise a ValueError if string is of invalid isoformat
            self.date = datetime.fromisoformat(self.date)

In [123]:
pycon_se_18 = PyCon("Stockholm", "2018-12-12")
pycon_se_18

PyCon(location='Stockholm', date=datetime.datetime(2018, 12, 12, 0, 0), year=2018)

In [117]:
from dataclasses import astuple
astuple(pycon_se_18)

('Stockholm', datetime.datetime(2018, 12, 12, 0, 0), 2018)

In [118]:
from dataclasses import asdict
asdict(pycon_se_18)

{'location': 'Stockholm',
 'date': datetime.datetime(2018, 12, 12, 0, 0),
 'year': 2018}

In [119]:
# Create a simple JSONEncoder

import json
from dataclasses import is_dataclass


class CustomEncoder(json.JSONEncoder):

   def default(self, obj):
        
        # Handle dataclasses
        if is_dataclass(obj):
            return asdict(obj)
        
        # Handle datetimes
        if isinstance(obj, datetime):
            return obj.isoformat()


'{"location": "Stockholm", "date": "2018-12-12T00:00:00", "year": 2018}'

Lets try it out!

In [121]:
json.dumps(pycon_se_18, cls=CustomEncoder)

'{"location": "Stockholm", "date": "2018-12-12T00:00:00", "year": 2018}'

And decoding?

In [120]:
json_payload = json.loads(json.dumps(pycon_se_18, cls=CustomEncoder))
PyCon(**json_payload)

PyCon(location='Stockholm', date=datetime.datetime(2018, 12, 12, 0, 0), year=2018)